In [ ]:
#| default_exp json_explorer

# json_explorer

> playing around with the google takeout json.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from collections import namedtuple

# make a datastructure to hold location data
Location = namedtuple("Location", ["timestamp", "latitude", "longitude", "accuracy"])


In [ ]:
#| export
from datetime import datetime

def get_timestamp(timestamp):
    """
    Google stores the timestamp in different formats, making this annoying
    """
    try:
        first_format = "%Y-%m-%dT%H:%M:%S.%fZ"
        timestamp=datetime.strptime(timestamp, first_format)
    except:
        second_format = "%Y-%m-%dT%H:%M:%SZ"
        timestamp=datetime.strptime(timestamp, second_format)
    return int(timestamp.timestamp())

# test data
test_string = "2022-06-24T19:38:55.633Z"
print(get_timestamp(test_string))
test_string2 = "2022-06-24T19:38:55Z"
print(get_timestamp(test_string2))

1656110335
1656110335


In [ ]:
show_doc(get_timestamp)

---

#### get_timestamp

>      get_timestamp (timestamp)

Google stores the timestamp in different formats, making this annoying

In [ ]:
#| export
import json
from pprint import pprint

file_to_open = "sample-data/Records.json"
def open_json_file(file_to_open): 
    with open(file_to_open, 'r') as f:
        return json.load(f)
    
    
output = open_json_file(file_to_open)
locations = [l for l in output["locations"]]

print(len(locations))
pprint(locations[1])
pprint(locations[-1])
        

672293
{'accuracy': 45,
 'deviceTag': 586642716,
 'latitudeE7': 446549213,
 'longitudeE7': -635836178,
 'source': 'WIFI',
 'timestamp': '2016-06-29T13:13:01.899Z'}
{'accuracy': 17,
 'activeWifiScan': {'accessPoints': [{'frequencyMhz': 2462,
                                      'mac': '229074580028182',
                                      'strength': -53},
                                     {'frequencyMhz': 5220,
                                      'mac': '229074580028183',
                                      'strength': -56},
                                     {'frequencyMhz': 2437,
                                      'isConnected': True,
                                      'mac': '189371410968206',
                                      'strength': -68},
                                     {'frequencyMhz': 2437,
                                      'mac': '145824591200902',
                                      'strength': -74},
                                     {'f

In [ ]:
#| export

# build up the location history
# due to the malformed google data, not all entries have latitudeE7/longitudeE7
# and the differing timestamps, best effort to parse those.

def build_location_history(locations):
    location_history = []
    for l in locations:
        if l.get("latitudeE7"):
            location_history.append(
                Location(get_timestamp(l["timestamp"]), l["latitudeE7"], l["longitudeE7"], l["accuracy"])
            )
    # sort by timestamp
    location_history.sort(key=lambda location: location.timestamp)
    return location_history


print(len(location_history))
print(location_history[0])
print(location_history[-1])

672291
Location(timestamp=1467216494, latitude=446549411, longitude=-635836042, accuracy=41)
Location(timestamp=1659139720, latitude=446579163, longitude=-636416645, accuracy=17)


In [ ]:
from nbdev.doclinks import nbdev_export
nbdev_export()